# logチェック

In [1]:
import os
import glob
import re
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
# 3桁区切りのカンマ形式での表示設定
pd.options.display.float_format = '{:,.0f}'.format
from scipy.stats import gmean

## logファイル読み込み

In [2]:
def extract_data_from_logs(log_contents):
    entries = re.split(r'(?=====================)', log_contents)
    data_list = []

    for entry in entries:
        if not entry.strip():
            continue

        # if not re.search(r'Processing input: (\d+)', entry):
        #     print(f"Skipped entry: {entry[:100]}...")  # 先頭100文字だけを表示
        #     continue

        data = {}

        data['input'] = re.search(r'Processing input: (\d+)', entry).group(1) if re.search(r'Processing input: (\d+)', entry) else None
        data['L'] = int(re.search(r'L=(\d+)', entry).group(1)) if re.search(r'L=(\d+)', entry) else None
        data['N'] = int(re.search(r'N=(\d+)', entry).group(1)) if re.search(r'N=(\d+)', entry) else None
        data['S'] = int(re.search(r'S=(\d+)', entry).group(1)) if re.search(r'S=(\d+)', entry) else None
        data['pass_flg'] = re.search(r'pass_flg=(\w+)', entry).group(1) == 'True' if re.search(r'pass_flg=(\w+)', entry) else None

        temp_search = re.search(r'max_temp(erture|erature)=(\d+)', entry)
        data['max_temperature'] = int(temp_search.group(2)) if temp_search else None

        data['interval'] = int(re.search(r'interval=(\d+)', entry).group(1)) if re.search(r'interval=(\d+)', entry) else None
        data['interval_num'] = int(re.search(r'interval_num=(\d+)', entry).group(1)) if re.search(r'interval_num=(\d+)', entry) else None

        data['Score'] = int(re.search(r'Score = (\d+)', entry).group(1)) if re.search(r'Score = (\d+)', entry) else None
        data['Number of wrong answers'] = int(re.search(r'Number of wrong answers = (\d+)', entry).group(1)) if re.search(r'Number of wrong answers = (\d+)', entry) else None
        data['Placement cost'] = int(re.search(r'Placement cost = (\d+)', entry).group(1)) if re.search(r'Placement cost = (\d+)', entry) else None
        data['Measurement cost'] = int(re.search(r'Measurement cost = (\d+)', entry).group(1)) if re.search(r'Measurement cost = (\d+)', entry) else None
        data['Measurement count'] = int(re.search(r'Measurement count = (\d+)', entry).group(1)) if re.search(r'Measurement count = (\d+)', entry) else None

        data_list.append(data)

    return data_list

In [3]:
# for path in glob.glob('./log*.txt'):
#     print(path)

In [4]:
df = pd.DataFrame()
for path in glob.glob('log*.txt'):
    # print(path)
    # サンプルのログファイルを読み込む
    with open(path, 'r') as file:
        log_contents = file.read()
    data_list = extract_data_from_logs(log_contents)
    # for data in data_list:
    #     print(data)
    _df = pd.DataFrame(data_list)
    _df['path'] = path    
    df = pd.concat([df, _df])

In [5]:
df.dropna(subset=['input'], inplace=True)
df

,input,L,N,S,pass_flg,max_temperature,interval,interval_num,Score,Number of wrong answers,Placement cost,Measurement cost,Measurement count,path
0,0000,50,95,36,None,71,71,2,"10,583,953",0,"4,769,266","4,579,000","3,990",log_049.txt
1,0001,35,66,289,None,567,567,2,"502,098",0,"196,252,750","2,811,600","2,442",log_049.txt
2,0002,23,89,324,None,636,636,2,"536,399",0,"182,234,622","4,094,000","3,560",log_049.txt
3,0003,35,61,4,None,32,8,5,"85,393,451",0,"137,750","933,300",854,log_049.txt
4,0004,33,81,529,None,"1,000","1,000",2,"150,886",0,"656,231,144","6,423,300","5,589",log_049.txt
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,0095,23,94,225,None,441,441,2,"1,066,622",0,"87,280,744","6,373,200","5,546",log_050.txt
96,0096,26,72,900,None,"1,000","1,000",2,"100,538",3,"497,636,566","11,524,800","10,000",log_050.txt
97,0097,33,87,529,None,"1,000","1,000",2,"145,796",0,"680,991,704","4,802,400","4,176",log_050.txt
98,0098,26,84,676,None,"1,000","1,000",2,"144,662",1,"541,394,928","11,520,800","10,000",log_050.txt


## 合計集計

In [6]:
df.groupby('path')['Score'].sum().sort_values(ascending=False).head(20)

path
log_048.txt   1,350,637,143
log_046.txt   1,334,709,950
log_050.txt   1,324,992,024
log_047.txt   1,320,661,568
log_049.txt   1,310,339,840
log_045.txt   1,292,642,653
log_042.txt   1,286,659,414
log_027.txt   1,282,359,324
log_026.txt   1,275,975,391
log_040.txt   1,272,054,760
log_025.txt   1,271,808,339
log_032.txt   1,270,705,461
log_039.txt   1,270,084,562
log_038.txt   1,265,478,503
log_037.txt   1,263,911,822
log_044.txt   1,260,475,496
log_028.txt   1,254,777,625
log_030.txt   1,240,661,142
log_043.txt   1,238,350,037
log_041.txt   1,226,692,653
Name: Score, dtype: float64

In [7]:
top_path = df.groupby('path')['Score'].sum().sort_values(ascending=False)[:15].index.tolist()
top_path.append('log.txt')

In [8]:
df[df['path'].isin(top_path)].pivot_table(values='Score', index=['S'], columns='path', aggfunc='mean')

path,log.txt,log_025.txt,log_026.txt,log_027.txt,log_032.txt,log_037.txt,log_038.txt,log_039.txt,log_040.txt,log_042.txt,log_045.txt,log_046.txt,log_047.txt,log_048.txt,log_049.txt,log_050.txt
S,,,,,,,,,,,,,,,,
1,"129,582,241","132,942,564","132,942,564","132,942,564","132,942,564","132,942,564","132,942,564","132,942,564","132,942,564","132,942,564","131,592,212","152,291,916","168,074,233","168,074,233","158,574,696","162,237,742"
4,"74,523,175","89,301,668","89,301,668","89,301,668","89,301,668","89,301,668","89,301,668","89,301,668","89,301,668","89,301,668","92,289,806","94,349,184","78,501,176","91,432,434","83,038,371","83,038,371"
9,"36,596,640","33,482,849","33,482,849","39,905,098","33,482,849","39,905,098","39,905,098","39,905,098","39,905,098","39,905,098","50,636,932","44,950,045","37,671,789","37,671,789","43,958,991","43,958,991"
16,"21,196,718","19,571,856","25,480,078","29,514,290","19,571,856","29,514,290","29,514,290","29,514,290","29,514,290","29,514,290","21,476,477","24,660,520","20,398,531","20,398,531","18,059,478","18,059,478"
25,"17,263,651","20,386,965","22,121,783","20,986,790","20,386,965","20,986,790","20,986,790","20,986,790","20,986,790","20,986,790","18,985,437","16,934,349","19,866,056","19,866,056","17,565,204","17,565,204"
36,"12,673,518","17,004,187","17,479,371","16,170,801","17,004,187","16,170,801","16,170,801","16,170,801","16,170,801","16,170,801","16,270,955","13,993,540","14,181,480","14,523,884","14,714,904","14,714,904"
49,"5,674,142","11,309,193","8,171,784","6,022,606","11,309,193","6,022,606","6,022,606","6,022,606","6,022,606","6,022,606","7,873,248","9,659,048","9,648,465","11,134,353","11,134,353","11,134,353"
81,"4,414,490","7,804,300","6,643,325","5,369,374","7,804,300","5,114,670","5,271,189","5,481,777","5,595,408","5,369,374","5,808,572","5,423,569","5,333,442","5,333,442","5,275,557","5,275,557"
100,"6,783,421","10,170,327","8,344,967","7,914,594","10,170,327","7,508,136","7,508,136","7,632,618","6,596,902","7,914,594","5,018,211","3,792,388","5,760,879","5,760,879","6,918,862","6,918,862"


In [9]:
df[df['path'].isin(top_path)].pivot_table(values='Score', index=['S', 'input'], columns='path', aggfunc='sum')

path          log.txt  log_025.txt  log_026.txt  log_027.txt  log_032.txt  \
S   input                                                                   
1   0010  141,721,750  154,918,668  154,918,668  154,918,668  154,918,668   
    0013  102,933,608   85,145,804   85,145,804   85,145,804   85,145,804   
    0034  142,726,650  144,757,604  144,757,604  144,757,604  144,757,604   
    0088  130,946,957  146,948,181  146,948,181  146,948,181  146,948,181   
4   0003   60,769,513   98,506,834   98,506,834   98,506,834   98,506,834   
    0055   88,276,837   80,096,501   80,096,501   80,096,501   80,096,501   
9   0030   31,237,076   29,325,325   29,325,325   26,692,021   29,325,325   
    0041   34,400,104   35,713,087   35,713,087   55,049,710   35,713,087   
    0079   40,664,689   33,752,518   33,752,518   42,266,606   33,752,518   
    0082   40,084,691   35,140,467   35,140,467   35,612,054   35,140,467   
16  0023   25,037,256   18,609,852   31,742,384   39,836,734   18,609,852   
    0036   15,578,334   20,910,118   22,456,469   18,924,448   20,910,118   
    0076   22,974,563   19,195,597   22,241,380   29,781,689   19,195,597   
25  0012   23,484,179   19,746,659   28,558,358   28,481,233   19,746,659   
    0018   16,325,257   22,120,674   21,872,936   19,445,228   22,120,674   
    0029   11,981,518   19,293,563   15,934,056   15,033,909   19,293,563   
36  0000    5,798,440    9,176,739    9,232,245    7,700,381    9,176,739   
    0007    8,210,747   12,836,569   10,927,477    8,275,539   12,836,569   
    0042   12,570,034   15,641,683   20,475,910   17,319,587   15,641,683   
    0043    9,933,583   13,410,586   15,243,169   13,818,118   13,410,586   
    0049   13,057,558   28,935,604   26,608,951   27,664,492   28,935,604   
    0061   22,359,270   18,196,687   20,833,256   23,782,718   18,196,687   
    0089    7,899,480   12,189,114   10,821,073   11,003,933   12,189,114   
    0099   21,559,029   25,646,511   25,692,886   19,801,643   25,646,511   
49  0074    5,674,142   11,309,193    8,171,784    6,022,606   11,309,193   
81  0011    2,120,184    4,200,893    3,184,383    2,366,887    4,200,893   
    0016    5,577,797    9,966,841    8,369,350    6,706,869    9,966,841   
    0017    7,391,802   10,372,336    9,496,509    9,106,998   10,372,336   
    0020    4,423,675    8,420,219    7,132,205    5,812,595    8,420,219   
    0056    3,791,836    8,042,582    5,925,324    3,948,612    8,042,582   
    0072    5,285,176    8,961,562    8,913,629    6,838,020    8,961,562   
    0077    2,310,958    4,665,664    3,481,877    2,805,636    4,665,664   
100 0048    7,061,138    9,559,073    8,187,011    9,147,503    9,559,073   
    0081    6,505,704   10,781,581    8,502,923    6,681,686   10,781,581   
121 0063    1,637,519    3,484,840    2,687,667    2,012,891    3,484,840   
    0071      798,529    1,896,837    1,225,299      969,724    1,896,837   
144 0014    1,823,815    4,186,306    3,020,304    2,039,877    4,186,306   
    0085    6,730,707    9,004,791    9,725,903    6,769,363    9,004,791   
169 0051    1,570,197    3,951,938    2,160,685    1,809,321    3,951,938   
    0083      442,945      946,473      639,108      472,832      946,473   
196 0009    6,287,470    9,433,922    8,176,257    8,163,388    9,433,922   
    0047    1,255,745    2,682,289    2,055,521    1,846,768    2,682,289   
225 0027    2,996,152    5,932,000    5,418,132    5,418,132    5,932,000   
    0031    1,284,079    2,925,988    2,235,080    2,235,080    2,925,988   
    0080      286,177      733,357      605,321      605,321      733,357   
    0095      971,444    2,254,118    1,834,989    1,834,989    2,254,118   
256 0032      205,833      496,181      496,181      496,181      496,181   
    0050      418,093    1,138,723    1,138,723    1,138,723    1,138,723   
    0054      230,636      620,127      620,127      620,127      620,127   
289 0001      261,610      876,105      876,105      876,105      876,105

## 幾何平均順

In [10]:
df.groupby('path').agg(geo_mean=('Score', lambda x: gmean(x))).sort_values('geo_mean', ascending=False)[:10]

,geo_mean
path,
log_042.txt,"1,835,705"
log_045.txt,"1,778,564"
log_046.txt,"1,727,189"
log_044.txt,"1,711,663"
log_040.txt,"1,618,016"
log_041.txt,"1,611,322"
log_039.txt,"1,527,734"
log_047.txt,"1,488,324"
log_048.txt,"1,481,702"


In [129]:
df[df['path'].isin(top_path)].pivot_table(values='Score', index=['S'], columns='path', aggfunc=gmean)

path,log.txt,log_025.txt,log_026.txt,log_027.txt,log_028.txt,log_032.txt,log_037.txt,log_038.txt,log_039.txt,log_040.txt,log_042.txt,log_044.txt,log_045.txt,log_046.txt,log_047.txt
S,,,,,,,,,,,,,,,
1,"164,231,699","129,424,993","129,424,993","129,424,993","119,599,205","129,424,993","129,424,993","129,424,993","129,424,993","129,424,993","129,424,993","147,597,970","130,290,654","147,867,270","164,231,699"
4,"74,844,327","88,825,969","88,825,969","88,825,969","96,680,496","88,825,969","88,825,969","88,825,969","88,825,969","88,825,969","88,825,969","94,528,091","92,289,478","94,141,153","74,844,327"
9,"36,514,970","33,384,585","33,384,585","38,564,058","42,067,735","33,384,585","38,564,058","38,564,058","38,564,058","38,564,058","38,564,058","24,802,492","50,087,068","44,195,243","36,514,970"
16,"19,729,804","19,547,911","25,121,616","28,211,018","31,866,745","19,547,911","28,211,018","28,211,018","28,211,018","28,211,018","28,211,018","21,277,326","21,277,326","24,168,832","19,729,804"
25,"19,522,830","20,350,175","21,510,750","20,268,172","19,602,945","20,350,175","20,268,172","20,268,172","20,268,172","20,268,172","20,268,172","18,922,137","18,922,137","16,528,192","19,522,830"
36,"13,971,519","15,891,384","16,246,197","14,705,034","12,993,872","15,891,384","14,705,034","14,705,034","14,705,034","14,705,034","14,705,034","15,064,922","15,064,922","13,315,529","13,971,519"
49,"9,648,465","11,309,193","8,171,784","6,022,606","4,978,829","11,309,193","6,022,606","6,022,606","6,022,606","6,022,606","6,022,606","7,873,248","7,873,248","9,659,048","9,648,465"
81,"5,260,177","7,412,595","6,147,895","4,855,837","3,794,762","7,412,595","4,707,026","5,010,868","5,230,058","5,393,938","4,855,837","4,526,667","5,664,485","5,162,515","5,260,177"
100,"5,755,211","10,151,942","8,343,472","7,817,976","6,575,736","10,151,942","7,425,971","7,425,971","7,544,670","6,491,912","7,817,976","4,904,877","4,904,877","3,788,782","5,755,211"


## Group集計

In [160]:
df.groupby('S')[['Score', 'Number of wrong answers', 'Placement cost', 'Measurement cost', 'Measurement count']].agg(['count', 'mean', 'std', 'min', 'max'])

Score                                                \
    count        mean        std        min         max   
S                                                         
1     164 129,170,451 41,184,426 49,178,716 246,935,531   
4      82  72,216,709 25,203,419  5,981,184 112,639,505   
9     164  27,133,250 12,071,271    932,359  60,191,361   
16    123  15,919,545  9,734,451    207,111  42,423,430   
25    123  13,794,248  6,853,884    146,540  28,558,358   
36    328  10,534,699  7,063,857      1,238  29,986,051   
49     41   6,724,631  2,276,391     61,641  11,959,703   
81    287   3,362,948  3,266,298          1  11,533,736   
100    82   4,362,982  4,032,057          1  10,852,902   
121    82   1,154,179  1,256,356          1   4,678,292   
144    82   2,821,376  3,043,775          1   9,725,903   
169    82     959,872  1,231,736         85   5,039,141   
196    82   3,049,484  3,496,187          3  10,728,940   
225   164   1,345,307  1,625,701          1   5,932,000   
256   123     363,725    389,255          8   1,853,828   
289   164     513,607    550,161         23   2,503,099   
324   164     631,720    725,607          2   2,713,804   
361   123     753,184    871,763          1   2,597,254   
400    41     371,794    368,729          8     956,094   
441   164     635,966    776,101          1   3,658,297   
484   164     912,714  1,031,341         28   4,268,072   
529   164     223,705    272,216          1   1,164,782   
576   123     433,145    505,341         68   1,725,459   
625    82     484,594    586,451          2   2,001,210   
676    82      52,305     66,059         12     312,708   
729   246     103,633    205,675          1     795,858   
784   205      54,912    117,923          1     499,190   
841   123     176,871    299,156         23     990,580   
900   246      28,843     53,873         15     227,303   

    Number of wrong answers                  Placement cost              \
                      count mean std min max          count        mean   
S                                                                         
1                       164    0   1   0   2            164     368,952   
4                        82    0   0   0   2             82     957,977   
9                       164    0   0   0   2            164   4,063,774   
16                      123    0   1   0   6            123   9,473,736   
25                      123    0   1   0   8            123  16,254,059   
36                      328    1   4   0  40            328  15,720,741   
49                       41    1   4   0  24             41  30,146,216   
81                      287   13  23   0  97            287  21,928,476   
100                      82   33  46   0  98             82  16,732,305   
121                      82   31  44   0  93             82  98,596,504   
144                      82   24  35   0  93             82  19,135,874   
169                      82   24  33   0  73             82 128,402,784   
196                      82   25  36   0  88             82  31,295,898   
225                     164   29  41   0  94            164  82,366,546   
256                     123   25  35   0  84            123 203,325,321   
289                     164   23  33   0  79            164 146,339,669   
324                     164   28  39   0  91            164 125,523,964   
361                     123   31  43   0  99            123 125,178,086   
400                      41   29  40   0  84             41 162,128,689   
441                     164   29  39   0  96            164 120,896,682   
484                     164   23  32   0  78            164  81,695,840   
529                     164   32  41   0  95            164 196,881,885   
576                     123   23  31   0  74            123 217,503,887   
625                      82   36  39   0  92             82  24,847,998   
676                      82   32  36   0  82             82 396,770,095   
729            

In [161]:
df[df['path']=='log.txt'].groupby('S')[['Score', 'Number of wrong answers', 'Placement cost', 'Measurement cost', 'Measurement count']].agg(['count', 'mean', 'std', 'min', 'max'])

Score                                                \
    count        mean        std        min         max   
S                                                         
1       4 132,942,564 32,162,255 85,145,804 154,918,668   
4       2  89,301,668 13,018,071 80,096,501  98,506,834   
9       4  39,905,098 11,943,640 26,692,021  55,049,710   
16      3  29,514,290 10,458,707 18,924,448  39,836,734   
25      3  20,986,790  6,854,921 15,033,909  28,481,233   
36      8  16,170,801  7,278,773  7,700,381  27,664,492   
49      1   6,022,606        NaN  6,022,606   6,022,606   
81      7   5,369,374  2,439,889  2,366,887   9,106,998   
100     2   7,914,594  1,743,596  6,681,686   9,147,503   
121     2   1,491,308    737,630    969,724   2,012,891   
144     2   4,404,620  3,344,252  2,039,877   6,769,363   
169     2   1,141,076    945,040    472,832   1,809,321   
196     2   5,005,078  4,466,525  1,846,768   8,163,388   
225     4   2,523,380  2,050,653    605,321   5,418,132   
256     3     751,677    340,873    496,181   1,138,723   
289     4   1,134,409    498,667    589,668   1,709,284   
324     4   1,544,809    902,723    614,677   2,713,804   
361     3   1,823,506  1,182,281    462,582   2,597,254   
400     1     956,094        NaN    956,094     956,094   
441     4   1,054,484    337,384    554,488   1,285,060   
484     4   2,001,180    967,866    729,199   2,881,684   
529     4     348,877    324,222    170,177     834,887   
576     3     484,899    470,365     84,925   1,003,098   
625     2   1,371,144     28,647  1,350,887   1,391,400   
676     2     119,460     81,756     61,650     177,270   
729     6     257,339    299,932     53,115     795,858   
784     5     180,932    171,540     41,699     477,961   
841     3     562,687    435,111    120,701     990,580   
900     6      98,557     71,942     41,960     227,303   

    Number of wrong answers                  Placement cost                \
                      count mean std min max          count          mean   
S                                                                           
1                         4    0   0   0   0              4       263,144   
4                         2    0   0   0   0              2       191,526   
9                         4    0   0   0   0              4       310,862   
16                        3    0   0   0   0              3       919,128   
25                        3    0   0   0   0              3     2,410,255   
36                        8    0   0   0   0              8     5,031,236   
49                        1    0 NaN   0   0              1    14,776,910   
81                        7    0   0   0   0              7    20,359,427   
100                       2    0   0   0   0              2     9,210,315   
121                       2    0   0   0   0              2    73,404,701   
144                       2    0   0   0   0              2    29,804,502   
169                       2    0   0   0   0              2   130,923,333   
196                       2    0   0   0   0              2    30,158,135   
225                       4    0   0   0   0              4    66,345,353   
256                       3    0   0   0   0              3   146,356,736   
289                       4    0   0   0   0              4    98,747,466   
324                       4    0   0   0   0              4    79,532,142   
361                       3    0   1   0   1              3    72,116,155   
400                       1    0 NaN   0   0              1    94,327,052   
441                       4    2   2   1   5              4    60,872,934   
484                       4    1   1   0   3              4    38,568,071   
529                       4    0   0   0   0              4   423,879,552   
576                       3    0   0   0   0              3   509,762,909   
625                       2    0   0   0   0              2    62,131,086   
676                       2    0   0

In [162]:
df.groupby(['S', 'input'])[['Score', 'Number of wrong answers', 'Placement cost', 'Measurement cost', 'Measurement count']].agg(['count', 'mean', 'std', 'min', 'max'])

Score                                                \
          count        mean        std        min         max   
S   input                                                       
1   0010     41 122,048,800 31,239,192 65,661,499 154,918,668   
    0013     41  97,098,560 25,299,111 49,178,716 144,780,658   
    0034     41 165,480,704 47,137,005 70,400,226 246,935,531   
    0088     41 132,053,740 24,653,327 82,689,048 159,782,696   
4   0003     41  84,252,317 19,261,345  5,981,184 104,744,508   
    0055     41  60,181,100 24,866,665 17,421,603 112,639,505   
9   0030     41  19,887,977  8,452,993    932,359  37,675,086   
    0041     41  32,943,797 14,679,592  3,821,411  60,191,361   
    0079     41  29,144,016 10,679,749  1,634,587  44,950,609   
    0082     41  26,557,210  9,994,658    933,840  42,530,135   
16  0023     41  18,179,001 12,746,755  1,707,732  42,423,430   
    0036     41  12,778,497  5,787,594    207,111  22,456,469   
    0076     41  16,801,136  8,769,171    590,020  34,704,340   
25  0012     41  15,598,699  9,143,376    423,038  28,558,358   
    0018     41  14,276,941  5,334,594    158,850  22,120,674   
    0029     41  11,507,104  4,720,031    146,540  19,293,563   
36  0000     41   5,862,824  2,873,610      9,392   9,872,035   
    0007     41   7,977,791  3,258,659     89,121  14,959,821   
    0042     41  12,822,718  4,472,289    272,873  20,475,910   
    0043     41   8,023,476  4,793,157      6,341  15,243,169   
    0049     41  15,155,364  9,828,455    659,924  28,935,604   
    0061     41  12,343,269  9,124,205      1,238  29,986,051   
    0089     41   7,029,203  4,193,555    158,998  12,940,133   
    0099     41  15,062,943  6,991,106    270,055  26,282,743   
49  0074     41   6,724,631  2,276,391     61,641  11,959,703   
81  0011     41   1,945,121  1,505,309        703   5,584,259   
    0016     41   4,053,670  3,912,501        561  11,533,736   
    0017     41   4,519,380  4,249,895          1  11,415,348   
    0020     41   3,353,511  3,402,787         79   8,931,374   
    0056     41   3,919,107  2,031,659      4,379   8,840,622   
    0072     41   3,862,032  3,776,850        115  10,931,837   
    0077     41   1,887,817  1,987,875          7   6,046,432   
100 0048     41   4,696,217  4,251,386          1  10,852,902   
    0081     41   4,029,747  3,823,543          1  10,781,581   
121 0063     41   1,526,943  1,496,692          1   4,678,292   
    0071     41     781,416    819,379          2   2,921,108   
144 0014     41   1,568,407  1,738,001        180   6,009,282   
    0085     41   4,074,346  3,538,607          1   9,725,903   
169 0051     41   1,503,878  1,504,894         85   5,039,141   
    0083     41     415,866    448,203        402   1,731,797   
196 0009     41   4,839,729  4,094,961      1,916  10,728,940   
    0047     41   1,259,239  1,188,828          3   4,138,624   
225 0027     41   2,672,427  2,312,854          1   5,932,000   
    0031     41   1,346,727  1,215,828        106   4,031,940   
    0080     41     340,486    334,611         18   1,193,345   
    0095     41   1,021,587    939,541          1   3,158,062   
256 0032     41     241,000    230,065        206     779,135   
    0050     41     548,769    520,767          8   1,853,828   
    0054     41     301,405    288,948        503     990,051   
289 0001     41     392,484    365,588        503   1,098,071   
    0022     41     602,444    557,377        981   1,793,317   
    0039     41     807,382    740,682         23   2,503,099   
    0053     41     252,121    242,490        206     741,860   
324 0002     41     752,677    702,297          3   2,057,260   
    0057     41   1,065,623  1,009,751         12   2,713,804   
    0065     41     253,975    242,201          2     672,016   
    0084     41     454,603    433,176        503   1,176,103   
361 0015     41     212,335    202,690          2     630,118   
    0091     41   1,075,004    992,733       

In [163]:
df[df['path']=='log.txt'].groupby(['S', 'input'])[['Score', 'Number of wrong answers', 'Placement cost', 'Measurement cost', 'Measurement count']].agg(['mean'])

Score Number of wrong answers Placement cost Measurement cost  \
                 mean                    mean           mean             mean   
S   input                                                                       
1   0010  154,918,668                       0        137,000          408,500   
    0013   85,145,804                       0        440,856          633,600   
    0034  144,757,604                       0        311,310          279,500   
    0088  146,948,181                       0        163,412          417,100   
4   0003   98,506,834                       0        292,958          622,200   
    0055   80,096,501                       0         90,094        1,058,400   
9   0030   26,692,021                       0        440,038        3,206,400   
    0041   55,049,710                       0        104,040        1,612,500   
    0079   42,266,606                       0        259,334        2,006,600   
    0082   35,612,054                       0        440,038        2,268,000   
16  0023   39,836,734                       0        241,246        2,169,000   
    0036   18,924,448                       0      1,889,970        3,294,200   
    0076   29,781,689                       0        626,168        2,631,600   
25  0012   28,481,233                       0      1,283,084        2,128,000   
    0018   19,445,228                       0      2,808,850        2,233,800   
    0029   15,033,909                       0      3,138,830        3,412,800   
36  0000    7,700,381                       0     10,473,372        2,413,000   
    0007    8,275,539                       0     10,180,406        1,803,400   
    0042   17,319,587                       0      4,048,210        1,625,600   
    0043   13,818,118                       0      4,096,876        3,040,000   
    0049   27,664,492                       0      1,699,142        1,815,600   
    0061   23,782,718                       0        629,534        3,475,200   
    0089   11,003,933                       0      6,798,260        2,189,400   
    0099   19,801,643                       0      2,324,086        2,626,000   
49  0074    6,022,606                       0     14,776,910        1,727,200   
81  0011    2,366,887                       0     40,010,690        2,138,900   
    0016    6,706,869                       0     12,423,288        2,386,800   
    0017    9,106,998                       0      6,509,768        4,370,800   
    0020    5,812,595                       0     14,349,820        2,754,200   
    0056    3,948,612                       0     23,321,356        1,904,000   
    0072    6,838,020                       0     12,619,118        1,905,000   
    0077    2,805,636                       0     33,281,946        2,260,600   
100 0048    9,147,503                       0      7,525,346        3,306,600   
    0081    6,681,686                       0     10,895,284        3,971,000   
121 0063    2,012,891                       0     46,571,810        3,008,000   
    0071      969,724                       0    100,237,592        2,784,600   
144 0014    2,039,877                       0     47,102,564        1,820,000   
    0085    6,769,363                       0     12,506,440        2,166,000   
169 0051    1,809,321                       0     52,386,962        2,782,400   
    0083      472,832                       0    209,459,704        1,932,000   
196 0009    8,163,388                       0     10,112,416        2,037,400   
    0047    1,846,768                       0     50,203,854        3,844,800   
225 0027    5,418,132                       0     13,036,548        5,320,000   
    0031    2,235,080                       0     40,925,940        3,715,200   
    0080      605,321                       0    161,215,070        3,886,800   
    0095    1,834,989                       0     50,203,854        4,192,400   
256 0032      496,181                       0  

## チューニング

In [44]:
df_params = pd.read_csv('ahc022_best_params.csv')

In [45]:
df_params['sample'] = df_params['sample'].apply(lambda x: f"{x:04d}")
df_params.rename(columns={'sample': "input"}, inplace=True)

In [46]:
df_params.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   input         100 non-null    object 
 1   L             100 non-null    int64  
 2   N             100 non-null    int64  
 3   S             100 non-null    int64  
 4   best_value    100 non-null    float64
 5   interval      100 non-null    int64  
 6   interval_num  100 non-null    int64  
dtypes: float64(1), int64(5), object(1)
memory usage: 5.6+ KB


In [47]:
df_params.sum()

input           0000000100020003000400050006000700080009001000...
L                                                            2910
N                                                            8005
S                                                           34501
best_value                                          1,111,152,372
interval                                                    19668
interval_num                                                10082
dtype: object

In [60]:
df_params.groupby(['S', 'input'])[['L', 'N', 'best_value', 'interval', 'interval_num']].sum()

L   N  best_value  interval  interval_num
S   input                                            
1   0010   22  95 125,232,620         7             5
    0013   43  96  50,518,369        13             2
    0034   36  65  60,577,425         1            57
    0088   24  97  95,412,018         1            58
4   0003   35  61  75,017,179        14             2
    0055   19  84  84,648,919        15             3
9   0030   40  96  30,653,784        12             5
    0041   19  75  68,234,049        14             5
    0079   30  79  43,100,030         2            30
    0082   40  81  37,645,500        18             3
16  0023   16  90  45,960,952        24             4
    0036   48  91  17,864,238         6            10
    0076   28  86  27,356,788        50             2
25  0012   24  76  22,480,721         1            90
    0018   39  73  17,120,128        11             7
    0029   40  79  14,350,497         5            16
36  0000   50  95   7,585,960        16             5
    0007   49  71   8,019,226         2            55
    0042   31  64  14,891,962        34             3
    0043   33  95  10,315,762        16             8
    0049   20  68  23,634,583         4            32
    0061   10  96  33,978,495        13            15
    0089   41  82   8,731,051        54             2
    0099   23  65  19,215,250        40             3
49  0074   43  68   7,399,283        14             9
81  0011   45  73   4,471,353         1           145
    0016   24  78   7,930,688         7            22
    0017   19  98   8,495,040         2           125
    0020   26  94   7,085,274         3            65
    0056   32  68   6,748,003        30             5
    0072   25  75   8,084,536         1           154
    0077   40  89   4,093,983        17             9
100 0048   15  99   9,563,975         6            44
    0081   18  95   7,646,886         1           220
121 0063   32  94   3,704,965        17            12
    0071   48  91   1,876,313         3            78
144 0014   26  65   4,499,782         5            52
    0085   12  95   8,191,573         7            52
169 0051   24  74   3,794,994         3            90
    0083   50  69   1,227,982         4            75
196 0009   10  61  10,522,311        14            23
    0047   23  89   3,170,516         1           353
225 0027   13  95   4,533,580         1           471
    0031   20  72   3,636,981         2           187
    0080   44  82   1,139,319        20            17
    0095   23  94   2,495,778         6            63
256 0032   48  70     832,601         2           186
    0050   31  85   1,503,803        32            13
    0054   43  71   1,001,237         6            63
289 0001   35  66   1,393,282         1           380
    0022   28  62   1,969,629         2           193
    0039   23  79   1,928,704        35            13
    0053   45  70     759,234        14            30
324 0002   23  89   1,452,119        93             6
    0057   19  82   2,277,301        44            11
    0065   42  93     561,011        13            41
    0084   31  67   1,227,094         6            78
361 0015   43  93     451,758        34            17
    0091   18  79   2,324,839         1           497
    0093   19  99   1,207,722       167             4
400 0078   32  86     602,346         1           680
441 0028   25  76     969,358       368             2
    0060   39  96     343,660         1           740
    0068   11  83   3,012,274         1           643
    0075   23  81   1,078,889         9            73
484 0021   17  62   1,773,570       117             6
    0038   13  71   2,530,736         2           335
    0066   32  79     522,013         8            90
    0090   17  63   1,705,678         1           671
529 0004   33  81     280,656       566             2
    0005   35  93     241,340       348             3
    0092   15  95   1,094,598         2           485
    0097   33  8

In [61]:
df[df['path']=="log.txt"].groupby(['S', 'input'])[['L', 'N', 'Score']].sum()

L  N       Score
S   input                  
1   0010  22 95 154,918,668
    0013  43 96  85,145,804
    0034  36 65 144,757,604
    0088  24 97 146,948,181
4   0003  35 61  98,506,834
    0055  19 84  80,096,501
9   0030  40 96  26,692,021
    0041  19 75  55,049,710
    0079  30 79  42,266,606
    0082  40 81  35,612,054
16  0023  16 90  39,836,734
    0036  48 91  18,924,448
    0076  28 86  29,781,689
25  0012  24 76  28,481,233
    0018  39 73  19,445,228
    0029  40 79  15,033,909
36  0000  50 95   7,700,381
    0007  49 71   8,275,539
    0042  31 64  17,319,587
    0043  33 95  13,818,118
    0049  20 68  27,664,492
    0061  10 96  23,782,718
    0089  41 82  11,003,933
    0099  23 65  19,801,643
49  0074  43 68   6,022,606
81  0011  45 73   2,366,887
    0016  24 78   6,706,869
    0017  19 98   9,106,998
    0020  26 94   5,812,595
    0056  32 68   3,948,612
    0072  25 75   6,838,020
    0077  40 89   2,805,636
100 0048  15 99   9,147,503
    0081  18 95   6,681,686
121 0063  32 94   2,012,891
    0071  48 91     969,724
144 0014  26 65   2,039,877
    0085  12 95   6,769,363
169 0051  24 74   1,809,321
    0083  50 69     472,832
196 0009  10 61   8,163,388
    0047  23 89   1,846,768
225 0027  13 95   5,418,132
    0031  20 72   2,235,080
    0080  44 82     605,321
    0095  23 94   1,834,989
256 0032  48 70     496,181
    0050  31 85   1,138,723
    0054  43 71     620,127
289 0001  35 66     876,105
    0022  28 62   1,362,579
    0039  23 79   1,709,284
    0053  45 70     589,668
324 0002  23 89   1,729,110
    0057  19 82   2,713,804
    0065  42 93     614,677
    0084  31 67   1,121,646
361 0015  43 93     462,582
    0091  18 79   2,597,254
    0093  19 99   2,410,681
400 0078  32 86     956,094
441 0028  25 76   1,157,340
    0060  39 96     554,488
    0068  11 83   1,285,060
    0075  23 81   1,221,049
484 0021  17 62   2,881,684
    0038  13 71   1,784,336
    0066  32 79     729,199
    0090  17 63   2,609,499
529 0004  33 81     195,045
    0005  35 93     170,177
    0092  15 95     834,887
    0097  33 87     195,399
576 0019  46 65      84,925
    0035  22 60     366,674
    0064  13 75   1,003,098
625 0037  10 85   1,391,400
    0046  10 92   1,350,887
676 0052  46 80      61,650
    0098  26 84     177,270
729 0045  11 87     795,858
    0059  16 68     430,938
    0062  46 71      53,115
    0067  34 73      97,308
    0073  31 98     113,699
    0094  46 70      53,118
784 0006  30 89     111,017
    0008  30 87     111,092
    0026  14 74     477,961
    0040  49 98      41,699
    0070  24 76     162,891
841 0025  10 65     990,580
    0044  29 79     120,701
    0087  13 67     576,781
900 0024  21 62     227,303
    0033  35 60      81,438
    0058  45 76      50,546
    0069  39 82      41,960
    0086  43 74      53,352
    0096  26 72     136,742

In [62]:
pd.concat([df_params.groupby(['S', 'input'])[['L', 'N', 'interval', 'interval_num', 'best_value']].sum(), df[df['path']=="log.txt"].groupby(['S', 'input'])['Score'].sum()], axis=1)

L   N  interval  interval_num  best_value       Score
S   input                                                        
1   0010   22  95         7             5 125,232,620 154,918,668
    0013   43  96        13             2  50,518,369  85,145,804
    0034   36  65         1            57  60,577,425 144,757,604
    0088   24  97         1            58  95,412,018 146,948,181
4   0003   35  61        14             2  75,017,179  98,506,834
    0055   19  84        15             3  84,648,919  80,096,501
9   0030   40  96        12             5  30,653,784  26,692,021
    0041   19  75        14             5  68,234,049  55,049,710
    0079   30  79         2            30  43,100,030  42,266,606
    0082   40  81        18             3  37,645,500  35,612,054
16  0023   16  90        24             4  45,960,952  39,836,734
    0036   48  91         6            10  17,864,238  18,924,448
    0076   28  86        50             2  27,356,788  29,781,689
25  0012   24  76         1            90  22,480,721  28,481,233
    0018   39  73        11             7  17,120,128  19,445,228
    0029   40  79         5            16  14,350,497  15,033,909
36  0000   50  95        16             5   7,585,960   7,700,381
    0007   49  71         2            55   8,019,226   8,275,539
    0042   31  64        34             3  14,891,962  17,319,587
    0043   33  95        16             8  10,315,762  13,818,118
    0049   20  68         4            32  23,634,583  27,664,492
    0061   10  96        13            15  33,978,495  23,782,718
    0089   41  82        54             2   8,731,051  11,003,933
    0099   23  65        40             3  19,215,250  19,801,643
49  0074   43  68        14             9   7,399,283   6,022,606
81  0011   45  73         1           145   4,471,353   2,366,887
    0016   24  78         7            22   7,930,688   6,706,869
    0017   19  98         2           125   8,495,040   9,106,998
    0020   26  94         3            65   7,085,274   5,812,595
    0056   32  68        30             5   6,748,003   3,948,612
    0072   25  75         1           154   8,084,536   6,838,020
    0077   40  89        17             9   4,093,983   2,805,636
100 0048   15  99         6            44   9,563,975   9,147,503
    0081   18  95         1           220   7,646,886   6,681,686
121 0063   32  94        17            12   3,704,965   2,012,891
    0071   48  91         3            78   1,876,313     969,724
144 0014   26  65         5            52   4,499,782   2,039,877
    0085   12  95         7            52   8,191,573   6,769,363
169 0051   24  74         3            90   3,794,994   1,809,321
    0083   50  69         4            75   1,227,982     472,832
196 0009   10  61        14            23  10,522,311   8,163,388
    0047   23  89         1           353   3,170,516   1,846,768
225 0027   13  95         1           471   4,533,580   5,418,132
    0031   20  72         2           187   3,636,981   2,235,080
    0080   44  82        20            17   1,139,319     605,321
    0095   23  94         6            63   2,495,778   1,834,989
256 0032   48  70         2           186     832,601     496,181
    0050   31  85        32            13   1,503,803   1,138,723
    0054   43  71         6            63   1,001,237     620,127
289 0001   35  66         1           380   1,393,282     876,105
    0022   28  62         2           193   1,969,629   1,362,579
    0039   23  79        35            13   1,928,704   1,709,284
    0053   45  70        14            30     759,234     589,668
324 0002   23  89        93             6   1,452,119   1,729,110
    0057   19  82        44            11   2,277,301   2,713,804
    0065   42  93        13            41     561,011     614,677
    0084   31  67         6            78   1,227,094   1,121,646
361 0015   43  93        34            17     451,758     462,582
    0091   18  79         1           497   2,324,83